When I first saw the osic pulmonary fibrosis competition I thought it looked like quite an interesting competition for its mix of tabular and image data and trying to produce a forecast based on that. This provides scope to try all sorts of things such as feature engineering, image embedding models and time series models. To get started I have focused on the tabular data only which as it turns out can get us quite far before even touching the images. I've tried to stay away from spending too much time on model selection and instead opted for feature experimentation.

But first a quick overview (disclaimer I'm not a medical expert so apologies if I get any of the medical stuff wrong). Pulmonary Fibrosis is a lung disease that causes the lungs to decline over time. The rate of decline can range between very rapid and very slow. We can measure the diseases progress through [FVC](https://lunginstitute.com/blog/fev1-and-fvc/#:~:text=The%20forced%20vital%20capacity%20(FVC,the%20severity%20of%20the%20condition)) (Forced Vital Capacity) which involves the patient taking a deep breath and blowing as hard as they can into a tube. The amount of air they blow out is the FVC measured in ml. 

So the challenge is when given a starting FVC for a patient to forecast the decline in the FVC value for all the upcoming weeks.

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import linear_model, ensemble
from sklearn.metrics import mean_squared_error, mean_absolute_error

import tensorflow as tf

from tqdm.notebook import tqdm

import os
from PIL import Image

## Load data

Let's begin by loading the data.

In [ ]:
train = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/train.csv')
test = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/test.csv')
submission = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/sample_submission.csv')

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
test.head()

In [ ]:
test.info()

So there's not a huge volume of tabular data. 1.5k of training examples with only 5 columns (weeks, percent, age, sex, smoking status) to construct features from.

## Merge datasets

I've learned recently that it is good practice to merge the train, validation and test sets at the start of a notebook. This ensures that exactly the same transformation is applied to every example. First though, I'll make sure there are no duplicates in the training dataset.

In [ ]:
train.drop_duplicates(keep=False, inplace=True, subset=['Patient','Weeks'])

Then form the submission dataset. The test dataset needs expanding out across the 146 weeks per patient that the competition requires. This can be achieved by joining it to the sample submission.

In [ ]:
submission['Patient'] = (
    submission['Patient_Week']
    .apply(
        lambda x:x.split('_')[0]
    )
)

submission['Weeks'] = (
    submission['Patient_Week']
    .apply(
        lambda x: int(x.split('_')[-1])
    )
)

submission =  submission[['Patient','Weeks', 'Confidence','Patient_Week']]

submission = submission.merge(test.drop('Weeks', axis=1), on="Patient")

In [ ]:
submission.head()

Mark each example in each dataset with the name of the dataset they come from. This enables me to quickly split the dataset back up into the three component pieces at the end of the notebook.

In [ ]:
train['Dataset'] = 'train'
test['Dataset'] = 'test'
submission['Dataset'] = 'submission'

Merge the datasets into one and reset the index.

In [ ]:
all_data = train.append([test, submission])

all_data = all_data.reset_index()
all_data = all_data.drop(columns=['index'])

In [ ]:
all_data.head()

## Quick data analysis

I think it's worth having a look at how the FVC (label) value declines for a sample of patients in the training dataset. Let's pick the first five in the data and plot the decline of the FVC.

In [ ]:
train_patients = train.Patient.unique()

In [ ]:
fig, ax = plt.subplots(5, 1, figsize=(10, 20))

for i in range(5):
    patient_log = train[train['Patient'] == train_patients[i]]

    ax[i].set_title(train_patients[i])
    ax[i].plot(patient_log['Weeks'], patient_log['FVC'])

So the decline is kinda linear as it does generally trend down over time. There are some spikes back up along the way which could cause a few issues. However this means that a simple linear model could have a good go at producing forecasts on this challenge as the main thing we need to do is predict the rate of decline for a patient. Like a trend line for these charts.

## Feature Engineering

There is some good scope for engineering new features for this model. 

### First FVC and First Week
Some useful features might be the first FVC recorded per patient and the week it was recorded in

In [ ]:
all_data['FirstWeek'] = all_data['Weeks']
all_data.loc[all_data.Dataset=='submission','FirstWeek'] = np.nan
all_data['FirstWeek'] = all_data.groupby('Patient')['FirstWeek'].transform('min')

In [ ]:
first_fvc = (
    all_data
    .loc[all_data.Weeks == all_data.FirstWeek][['Patient','FVC']]
    .rename({'FVC': 'FirstFVC'}, axis=1)
    .groupby('Patient')
    .first()
    .reset_index()
)

all_data = all_data.merge(first_fvc, on='Patient', how='left')

In [ ]:
all_data.head()

### Weeks Passed
This feature measures how many weeks have passed since the patients first FVC reading.

In [ ]:
all_data['WeeksPassed'] = all_data['Weeks'] - all_data['FirstWeek']

In [ ]:
all_data.head()

### Patient height

Apparently the height of a patient is an important variable when predicting FVC. Maybe tall people have larger lungs and thus more air to exhale. Thanks to Srikanth Potukuchi whose [notebook](https://www.kaggle.com/srikanthpotukuchi/osic-random-forest-new-height-extracted) showed me how to estimate a patients height using their FVC.

In [ ]:
def calculate_height(row):
    if row['Sex'] == 'Male':
        return row['FirstFVC'] / (27.63 - 0.112 * row['Age'])
    else:
        return row['FirstFVC'] / (21.78 - 0.101 * row['Age'])

all_data['Height'] = all_data.apply(calculate_height, axis=1)

In [ ]:
all_data.head()

### Categorical columns

The sex and smoking status columns are categorical columns that need some transformation to turn them into numbers. Pandas get dummies makes this easy to achieve.

In [ ]:
all_data = pd.concat([
    all_data,
    pd.get_dummies(all_data.Sex),
    pd.get_dummies(all_data.SmokingStatus)
], axis=1)

all_data = all_data.drop(columns=['Sex', 'SmokingStatus'])

In [ ]:
all_data.head()

### Scale features

Now scale all the features to get them onto the same range of numbers (0-1).

In [ ]:
def scale_feature(series):
    return (series - series.min()) / (series.max() - series.min())

all_data['Weeks'] = scale_feature(all_data['Weeks'])
all_data['Percent'] = scale_feature(all_data['Percent'])
all_data['Age'] = scale_feature(all_data['Age'])
all_data['FirstWeek'] = scale_feature(all_data['FirstWeek'])
all_data['FirstFVC'] = scale_feature(all_data['FirstFVC'])
all_data['WeeksPassed'] = scale_feature(all_data['WeeksPassed'])
all_data['Height'] = scale_feature(all_data['Height'])

Specify what columns will be used as features. This is for easy filtering of the datasets later.

In [ ]:
feature_columns = [
    'Percent',
    'Age',
    'FirstWeek',
    'FirstFVC',
    'WeeksPassed',
    'Height',
    'Female',
    'Male', 
    'Currently smokes',
    'Ex-smoker',
    'Never smoked',
]

### Split dataframe

Split the data back into the three dataframes they started as.

In [ ]:
train = all_data.loc[all_data.Dataset == 'train']
test = all_data.loc[all_data.Dataset == 'test']
submission = all_data.loc[all_data.Dataset == 'submission']

And take a look at the features that will be used to train the model.

In [ ]:
train[feature_columns].head()

## Model

As I mentioned at the start of the notebook I didn't intend to experiment too much with model selection in this notebook so I have stuck with a simple Linear Regressor from SKLearn. While these non-deep models don't usually produce as accurate results as deep models they are super fast to train and easy to evaluate. This is great for notebooks focussing primarily on features.

In [ ]:
model = linear_model.HuberRegressor(max_iter=200)

With the model loaded, insert the features and labels for training.

In [ ]:
model.fit(train[feature_columns], train['FVC'])

Make predictions

In [ ]:
predictions = model.predict(train[feature_columns])

## Evaluate

Let's begin by having a look at the models weights. This gives us a good indication of what features are driving the models predictions.

In [ ]:
plt.bar(train[feature_columns].columns.values, model.coef_)
plt.xticks(rotation=90)
plt.show()

While mean squared error isn't the competition metric it is a simple loss metric to help understand how close the models predictions are to the actual labels. The limitation of this error number though is that it can't be too close to zero as that would indicate over-fitting a model that should only be producing a trend line.

In [ ]:
mse = mean_squared_error(
    train['FVC'],
    predictions,
    squared=False
)

mae = mean_absolute_error(
    train['FVC'],
    predictions
)

print('MSE Loss: {0:.2f}'.format(mse))
print('MAE Loss: {0:.2f}'.format(mae))

Found code for competition metric [here](https://www.kaggle.com/titericz/tabular-simple-eda-linear-model#Calculate-competition-metric)

In [ ]:
def competition_metric(trueFVC, predFVC, predSTD):
    clipSTD = np.clip(predSTD, 70 , 9e9)  
    deltaFVC = np.clip(np.abs(trueFVC - predFVC), 0 , 1000)  
    return np.mean(-1 * (np.sqrt(2) * deltaFVC / clipSTD) - np.log(np.sqrt(2) * clipSTD))
    

print(
    'Competition metric: ', 
    competition_metric(train['FVC'].values, predictions, 285) 
)

Let's also include a scatterplot and histogram to see an overview of how close the predictions are to the labels.

In [ ]:
train['prediction'] = predictions

In [ ]:
plt.scatter(predictions, train['FVC'])

plt.xlabel('predictions')
plt.ylabel('FVC (labels)')
plt.show()

In [ ]:
delta = predictions - train['FVC']
plt.hist(delta, bins=20)
plt.show()

Finally take the first five patients as a sample and compare the true FVC readings against the models predicted FVC readings.

In [ ]:
fig, ax = plt.subplots(5, 1, figsize=(10, 20))

for i in range(5):
    patient_log = train[train['Patient'] == train_patients[i]]

    ax[i].set_title(train_patients[i])
    ax[i].plot(patient_log['WeeksPassed'], patient_log['FVC'], label='truth')
    ax[i].plot(patient_log['WeeksPassed'], patient_log['prediction'], label='prediction')
    ax[i].legend()

## Submission

With the model trained we can use it to make predictions. As the submission dataframe went through the same transformation as the train dataset it is ready for the model to infer on.

In [ ]:
submission[feature_columns].head()

Add the predictions to the submission dataframe.

In [ ]:
sub_predictions = model.predict(submission[feature_columns])
submission['FVC'] = sub_predictions

Plot the forecasted FVC for five patients in the test set.

In [ ]:
test_patients = list(submission.Patient.unique())
fig, ax = plt.subplots(5, 1, figsize=(10, 20))

for i in range(5):
    patient_log = submission[submission['Patient'] == test_patients[i]]

    ax[i].set_title(test_patients[i])
    ax[i].plot(patient_log['WeeksPassed'], patient_log['FVC'])

Drop the surplus columns not needed for submission and add a dummy confidence column.

In [ ]:
submission = submission[['Patient_Week', 'FVC']]

submission['Confidence'] = 285

And write the submission to file.

In [ ]:
submission.to_csv('submission.csv', index=False)

In [ ]:
submission.head()